In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
from stellargraph import StellarGraph
from stellargraph import datasets
from IPython.display import display, HTML


import pandas as pd
import numpy as np
from stellargraph import StellarGraph
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph

from stellargraph import datasets

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
import json

In [ ]:
filehandler = open("../data/sample_data/social_network.pkl","rb")
dat = pickle.load(filehandler)
filehandler.close()
print(type(dat))
print(dat.shape)

In [ ]:
df = dat.sample(frac=1)
df_without_labels = df.copy()
df_without_labels = df_without_labels.drop(columns=['label'])
df.columns
df_check = df[df.label == -1]

In [ ]:
print(df.columns)
print(df_without_labels.columns)

In [ ]:
df['has_bottleneck'] = 1
df.loc[df.label == -1,'has_bottleneck'] = 0
df_graph_class = df.copy()
df_graph_class['has_bottleneck'] = 1
df_graph_class.loc[df_graph_class.label == -1,'has_bottleneck'] = 0
df_graph_class = df_graph_class.drop(columns = ['label'])


In [ ]:
node_indices = [str(i+1) for i in range(len(df_without_labels.columns))]

In [ ]:
graph_edges = pd.DataFrame(
    {"source": ["1", "1", "1", "1", "2", "3", "4", "4",  "4",  "5",  "6",  "6",  "6",  "7",  "7",  "7",  "8",  "9", "10", "10" ,"10", "11", "11", "11", "12", "12", "12", "13", "13", "13"], 
     "target": ["2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "14", "15", "16", "23", "24", "25", "12", "13", "17", "18", "19", "26", "27", "28", "20", "21", "22", "29", "30", "31"]}
)
graph_edges

In [ ]:
row = df.iloc[0]
label = row['has_bottleneck']
node_features = row.drop(labels = ['label', 'has_bottleneck'])
graph_nodes = pd.DataFrame({"x": node_features, "index" : node_indices})

In [ ]:
node_indices

In [ ]:
# graph = StellarGraph(graph_nodes, graph_edges)
# print(graph.info())

dataset = datasets.PROTEINS()
display(HTML(dataset.description))
graphs, graph_labels = dataset.load()

In [ ]:
graph_named = StellarGraph(
    edges=graph_edges, node_type_default="default", edge_type_default="line"
)
print(graph_named.info())

In [ ]:
print(graphs[0].info())

In [ ]:
print(graphs[1].info())

In [ ]:
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in graphs],
    columns=["nodes", "edges"],
)
summary.describe().round(1)

In [ ]:
graph_labels.value_counts().to_frame()

In [ ]:
graph_labels = pd.get_dummies(graph_labels, drop_first=True)

In [ ]:
k = 35  # the number of rows for the output tensor
layer_sizes = [32, 32, 32, 1]
generator = PaddedGraphGenerator(graphs=graphs)

dgcnn_model = DeepGraphCNN(
    layer_sizes=layer_sizes,
    activations=["tanh", "tanh", "tanh", "tanh"],
    k=k,
    bias=False,
    generator=generator,
)
x_inp, x_out = dgcnn_model.in_out_tensors()

In [ ]:
x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
x_out = MaxPool1D(pool_size=2)(x_out)

x_out = Conv1D(filters=32, kernel_size=5, strides=1)(x_out)

x_out = Flatten()(x_out)

x_out = Dense(units=128, activation="relu")(x_out)
x_out = Dropout(rate=0.5)(x_out)

predictions = Dense(units=1, activation="sigmoid")(x_out)

In [ ]:
model = Model(inputs=x_inp, outputs=predictions)

model.compile(
    optimizer=Adam(lr=0.0001), loss=binary_crossentropy, metrics=["acc"],
)

In [ ]:
train_graphs, test_graphs = model_selection.train_test_split(
    graph_labels, train_size=0.9, test_size=None, stratify=graph_labels,
)

In [ ]:
gen = PaddedGraphGenerator(graphs=graphs)

train_gen = gen.flow(
    list(train_graphs.index - 1),
    targets=train_graphs.values,
    batch_size=50,
    symmetric_normalization=False,
)

test_gen = gen.flow(
    list(test_graphs.index - 1),
    targets=test_graphs.values,
    batch_size=1,
    symmetric_normalization=False,
)

In [ ]:
epochs = 100

In [ ]:
history = model.fit(
    train_gen, epochs=epochs, verbose=1, validation_data=test_gen, shuffle=True,
)
sg.utils.plot_history(history)
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))
    
history_dict = history.history
json.dump(history_dict, open("history", 'w'))
sg.utils.plot_history(history)
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
import stellargraph as sg
import os
import sys
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from stellargraph.layer import GCN_LSTM
dataset = sg.datasets.METR_LA()

In [ ]:
speed_data, sensor_dist_adj = dataset.load()
num_nodes, time_len = speed_data.shape
print("No. of sensors:", num_nodes, "\nNo of timesteps:", time_len)

In [ ]:
def train_test_split(data, train_portion):
    time_len = data.shape[1]
    train_size = int(time_len * train_portion)
    train_data = np.array(data.iloc[:, :train_size])
    test_data = np.array(data.iloc[:, train_size:])
    return train_data, test_data

In [ ]:
train_rate = 0.8

In [ ]:
train_data, test_data = train_test_split(speed_data, train_rate)
print("Train data: ", train_data.shape)
print("Test data: ", test_data.shape)

In [ ]:
def scale_data(train_data, test_data):
    max_speed = train_data.max()
    min_speed = train_data.min()
    train_scaled = (train_data - min_speed) / (max_speed - min_speed)
    test_scaled = (test_data - min_speed) / (max_speed - min_speed)
    return train_scaled, test_scaled

In [ ]:
train_scaled, test_scaled = scale_data(train_data, test_data)

In [ ]:
seq_len = 10
pre_len = 12

In [ ]:
def sequence_data_preparation(seq_len, pre_len, train_data, test_data):
    trainX, trainY, testX, testY = [], [], [], []

    for i in range(train_data.shape[1] - int(seq_len + pre_len - 1)):
        a = train_data[:, i : i + seq_len + pre_len]
        trainX.append(a[:, :seq_len])
        trainY.append(a[:, -1])

    for i in range(test_data.shape[1] - int(seq_len + pre_len - 1)):
        b = test_data[:, i : i + seq_len + pre_len]
        testX.append(b[:, :seq_len])
        testY.append(b[:, -1])

    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)

    return trainX, trainY, testX, testY

In [ ]:
trainX, trainY, testX, testY = sequence_data_preparation(
    seq_len, pre_len, train_scaled, test_scaled
)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
gcn_lstm = GCN_LSTM(
    seq_len=seq_len,
    adj=sensor_dist_adj,
    gc_layer_sizes=[16, 10],
    gc_activations=["relu", "relu"],
    lstm_layer_sizes=[200, 200],
    lstm_activations=["tanh", "tanh"],
)

In [ ]:
x_input, x_output = gcn_lstm.in_out_tensors()

In [ ]:
model = Model(inputs=x_input, outputs=x_output)

In [ ]:
model.compile(optimizer="adam", loss="mae", metrics=["mse"])

In [ ]:
history = model.fit(
    trainX,
    trainY,
    epochs=100,
    batch_size=60,
    shuffle=True,
    verbose=0,
    validation_data=[testX, testY],
)

In [ ]:
model.summary()

In [ ]:
print(
    "Train loss: ",
    history.history["loss"][-1],
    "\nTest loss:",
    history.history["val_loss"][-1],
)
sg.utils.plot_history(history)
ythat = model.predict(trainX)
yhat = model.predict(testX)

In [ ]:
df_without_labels.columns